## Notebook for ASHRAE on Kaggle 
https://www.kaggle.com/c/ashrae-energy-prediction/

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
test = pd.read_csv('data/test.csv')
weather_test = pd.read_csv('data/weather_test.csv')

In [ ]:
train = pd.read_csv('data/train.csv')

In [ ]:
metadata = pd.read_csv('data/building_metadata.csv')
weather_train = pd.read_csv('data/weather_train.csv')

In [ ]:
train.head()

In [ ]:
metadata.head()

In [ ]:
weather_train.head()

In [ ]:
merged_df = train.merge(metadata, on = 'building_id',how = 'left')

In [ ]:
merged_df.head()

In [ ]:
weather_train['site_id'].nunique()

In [ ]:
merged_df = merged_df.merge(weather_train,on = ['timestamp','site_id'], how = 'left')

In [ ]:
merged_df.head()

In [ ]:
merged_df.corr()['meter_reading'].sort_values(ascending = False).head(10)

In [ ]:
merged_df['meter_reading'].value_counts()

In [ ]:
merged_df['floor_count'].value_counts()

In [ ]:
merged_df['primary_use'].value_counts() 	

In [ ]:
usage = {'Education':0,'Office':1,'Entertainment/public assembly':2,
         'Lodging/residential':3,'Public services':4,'Healthcare':5,
         'Other':6,'Parking':7,'Manufacturing/industrial':8,
        'Food sales and service':9,'Retail':10,'Warehouse/storage':11,
        'Services':12,'Technology/science':13,'Utility':14,
        'Religious worship':15}

In [ ]:
usage_func = lambda x: usage[x]

In [ ]:
merged_df['primary_use'] = merged_df['primary_use'].apply(usage_func)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(np.log(merged_df['square_feet']))

In [ ]:
merged_df['year_built'] = merged_df['year_built']-1900
merged_df['square_feet'] = np.log(merged_df['square_feet'])

In [ ]:
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]

In [ ]:
holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
            "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
            "2017-01-01", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
            "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
            "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
            "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
            "2019-01-01"]

In [ ]:
merged_df["timestamp"] = pd.to_datetime(merged_df["timestamp"])
merged_df["is_holiday"] = (merged_df['timestamp'].dt.date.astype("str").isin(holidays)).astype(int)
merged_df["weekday"] = merged_df["timestamp"].dt.weekday
merged_df["hour"] = merged_df["timestamp"].dt.hour
merged_df["weekday"] = merged_df['weekday'].astype(np.uint8)
merged_df["hour"] = merged_df['hour'].astype(np.uint8)

In [ ]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df

In [ ]:
merged_df = average_imputation(merged_df, 'wind_speed')
merged_df = average_imputation(merged_df, 'wind_direction')

beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    merged_df.loc[(merged_df['wind_speed']>=item[1]) & (merged_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [ ]:
merged_df['meter_reading_log1p'] = np.log1p(merged_df['meter_reading'])
df_group = merged_df.groupby('building_id')['meter_reading_log1p']

In [ ]:
merged_df['wind_direction'] = merged_df['wind_direction'].apply(degToCompass)
merged_df['beaufort_scale'] = merged_df['beaufort_scale'].astype(np.uint8)
merged_df["wind_direction"] = merged_df['wind_direction'].astype(np.uint8)
merged_df["meter"] = merged_df['meter'].astype(np.uint8)
merged_df["site_id"] = merged_df['site_id'].astype(np.uint8)

building_median = df_group.median().astype(np.float16)
merged_df['building_median'] = merged_df['building_id'].map(building_median)

In [ ]:
merged_df.head()

In [ ]:
from sklearn.metrics import roc_auc_score
import lightgbm
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score)

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
merged_df_c = merged_df.copy()

In [ ]:
del merged_df_c['timestamp']

In [ ]:
import gc
del train, weather_train
gc.collect()

In [ ]:
## REducing memory
test = reduce_mem_usage(test)

weather_test = reduce_mem_usage(weather_test)
meta = reduce_mem_usage(metadata)

merged_df = reduce_mem_usage(merged_df)
merged_df_c = reduce_mem_usage(merged_df_c)

In [ ]:
y = np.log1p(merged_df_c['meter_reading'])#.values
del merged_df_c['meter_reading']
x = merged_df_c.select_dtypes(exclude = ['object'])#.values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

In [ ]:
categoricals = ["site_id",
                "building_id","building_median",
                "primary_use", "hour", "weekday", "meter",  "wind_direction", "is_holiday"]

drop_cols = ["sea_level_pressure", "wind_speed"]

numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage",
              "dew_temperature", 'precip_depth_1_hr', 'floor_count', 'beaufort_scale']

feat_cols = categoricals + numericals

x = x.drop(drop_cols, axis = 1)

In [ ]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.25,
            'subsample_freq': 1,
            'learning_rate': 0.4,
            'num_leaves': 42,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

folds = 4
seed = 111

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

models = []
for train_index, val_index in kf.split(x, x['building_id']):
    train_X = x[feat_cols].iloc[train_index]
    val_X = x[feat_cols].iloc[val_index]
    train_y = y.iloc[train_index]
    val_y = y.iloc[val_index]
    lgb_train = lightgbm.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lightgbm.Dataset(val_X, val_y, categorical_feature=categoricals)
    gbm = lightgbm.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)


In [ ]:
def visualize_feature_importance(model,columns = feat_cols):
    feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(),columns), reverse=True), columns=["Value","Feature"])
    plt.figure(figsize=(16,16))
    importance_bar = sns.barplot(data=feature_imp, x='Value', y='Feature')
    plt.show()

In [ ]:
import seaborn as sns

In [ ]:
visualize_feature_importance(models[0])

In [ ]:
visualize_feature_importance(models[2])

In [ ]:
del x, train_X, val_X, lgb_train, lgb_eval, train_y, val_y, y
gc.collect()

In [ ]:
test_merged = test.merge(metadata, left_on = "building_id", right_on = "building_id", how = "left")

test_merged["primary_use"] = test_merged["primary_use"].apply(usage_func)

test_merged = test_merged.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")

In [ ]:
building_median = df_group.median().astype(np.float16)
test_merged['building_median'] = test_merged['building_id'].map(building_median)

In [ ]:
test_merged["timestamp"] = pd.to_datetime(test_merged["timestamp"])
test_merged["is_holiday"] = (test_merged['timestamp'].dt.date.astype("str").isin(holidays)).astype(int)
test_merged["hour"] = test_merged["timestamp"].dt.hour
test_merged["weekday"] = test_merged["timestamp"].dt.weekday
test_merged["weekday"] = test_merged['weekday'].astype(np.uint8)
test_merged["hour"] = test_merged['hour'].astype(np.uint8)
test_merged['year_built'] = test_merged['year_built']-1900
test_merged['square_feet'] = np.log(test_merged['square_feet'])

test_merged = average_imputation(test_merged, 'wind_speed')
test_merged = average_imputation(test_merged, 'wind_direction')

for item in beaufort:
    test_merged.loc[(test_merged['wind_speed']>=item[1]) & (test_merged['wind_speed']<item[2]), 'beaufort_scale'] = item[0]
test_merged['wind_direction'] = test_merged['wind_direction'].apply(degToCompass)

test_merged['wind_direction'] = test_merged['wind_direction'].apply(degToCompass)
test_merged['beaufort_scale'] = test_merged['beaufort_scale'].astype(np.uint8)
test_merged["wind_direction"] = test_merged['wind_direction'].astype(np.uint8)
test_merged["meter"] = test_merged['meter'].astype(np.uint8)
test_merged["site_id"] = test_merged['site_id'].astype(np.uint8)

test_merged = test_merged[feat_cols]

In [ ]:
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    res.append(np.expm1(sum([model.predict(test_merged.iloc[i:i+step_size]) for model in models])/folds))
    i+=step_size

In [ ]:
res = np.concatenate(res)

In [ ]:
res

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission.csv', index=False)
submission